In [95]:
from flask import Flask, render_template, request, Markup
import pandas as pd

app = Flask(__name__)

# Read the CSV file directly from a URL
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv'
df = pd.read_csv(url)

@app.route('/', methods=['GET'])
def index():
    # Get column names for the dropdown list
    columns = df.columns.tolist()
    # List of available aggregation operations
    operations = ['sum', 'mean', 'max', 'min', 'count']
    return render_template('index.html', columns=columns, operations=operations)

@app.route('/apply', methods=['POST'])
def apply():
    # Get the selected group keys
    groupby_columns = request.form.getlist('groupby[]')

    # Validate that at least one group key is selected
    if not groupby_columns:
        return render_template('index.html', error='Please select at least one group key.', columns=df.columns.tolist(), operations=['sum', 'mean', 'max', 'min', 'count'])

    # Filter the DataFrame based on the selected columns
    selected_columns = groupby_columns + request.form.getlist('columns[]')
    df_filtered = df[selected_columns]

    # Group by the selected columns
    df_grouped = df_filtered.groupby(groupby_columns)

    # Get the selected operations
    operation_names = request.form.getlist('operation[]')

    # Perform the selected aggregation operations
    df_grouped = df_grouped.agg(operation_names)

    # Convert DataFrame to an HTML table
    table = df_grouped.to_html()

    return render_template('index.html', table=Markup(table), columns=df.columns.tolist(), operations=['sum', 'mean', 'max', 'min', 'count'])

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Jun/2023 17:11:32] "GET /apply HTTP/1.1" 405 -
127.0.0.1 - - [02/Jun/2023 17:11:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2023 17:11:38] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2023 17:11:46] "POST /apply HTTP/1.1" 200 -
